In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
!pip install plotly

In [36]:
df = pd.read_csv("12.csv")
# we must change delivery_cost from Rial to Toman!
df.head()

,id,user_id,coupon_discount,paid_at,amount,quantity,price,delivery_cost
0,243598,138523,NaN,2020-10-09T07:07:49,59000.0,1,50000.0,90000
1,2217990,1580455,NaN,2020-12-28T03:37:01,30000.0,2,15000.0,0
2,1364113,1017264,25000.0,2020-10-07T12:56:00,60000.0,1,20000.0,0
3,1364113,1017264,25000.0,2020-10-07T12:56:00,60000.0,1,25000.0,0
4,1364113,1017264,25000.0,2020-10-07T12:56:00,60000.0,2,20000.0,0


In [37]:
monetary_df = df.groupby('user_id')['amount'].sum()
# mamad = df.loc[df['user_id'] == 15]
# mamad
monetary_df.columns = ['user_id', 'Monetary']
monetary_df.head(10)

user_id
13       2939330.0
15      10386540.0
16        324450.0
20        678900.0
83       2119800.0
112       301000.0
113    127209775.0
271        32200.0
277       457000.0
321       567640.0
Name: amount, dtype: float64

In [46]:
import datetime as dt
now = dt.date(2021,1,1)
df['Date'] = pd.DatetimeIndex(df.paid_at).date
recency_df = df.groupby(['user_id'],as_index=False)['Date'].max()
recency_df.columns = ['user_id','LastP']
recency_df.sort_values(by=['LastP']).head()


,user_id,LastP
7784,190901,2020-10-02
51266,987129,2020-10-02
51267,987139,2020-10-02
46600,945628,2020-10-02
51270,987144,2020-10-02


In [50]:
recency_df['Recency'] = recency_df.LastP.apply(lambda x : (now - x).days)
recency_df.sort_values(by=['Recency']).head()

,user_id,LastP,Recency
199115,1750068,2020-12-31,1
139439,1407848,2020-12-31,1
139451,1407919,2020-12-31,1
139498,1408286,2020-12-31,1
139511,1408392,2020-12-31,1


In [54]:
frequency_df = df.copy()
#frequency_df.drop_duplicates(subset=['user_id','InvoiceNo'], keep="first", inplace=True) 
frequency_df = frequency_df.groupby('user_id',as_index=False)['quantity'].count()
frequency_df.columns = ['user_id','Frequency']
frequency_df.sort_values(by=["Frequency"]).tail()

,user_id,Frequency
8853,210081,258
50211,980257,260
46516,944760,267
37589,815767,293
3279,83049,757


In [ ]:
rf = recency_df.merge(frequency_df,left_on='user_id',right_on='user_id')

#combibe rf frame with monetary values..

rfm = rf.merge(monetary_df,left_on='user_id',right_on='user_id')

rfm.set_index('user_id',inplace=True)
rfm.head()

In [ ]:
import plotly.graph_objects as go
import numpy as np

N = 100000
fig = go.Figure(data=go.Scattergl(
    x = rfm['Frequency'],
    y = rfm['amount'],
    mode='markers',
    marker=dict(
        color=np.random.randn(N),
        colorscale='Viridis',
        line_width=1
    ),
))

fig.update_xaxes(title_text='Frequency')
fig.update_yaxes(title_text='Monetization')

# fig.show()